In [1]:
### 8 April Assignment Solution

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC,SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import re

In [3]:
df_new = pd.read_csv('Bengaluru_new_cleaned.csv')
df_new.head()

,Unnamed: 0,location,size,total_sqft,bath,price
0,0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,2,Uttarahalli,3 BHK,1440,2.0,62.00
3,3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,4,Kothanur,2 BHK,1200,2.0,51.00


In [4]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13246 entries, 0 to 13245
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  13246 non-null  int64  
 1   location    13246 non-null  object 
 2   size        13246 non-null  object 
 3   total_sqft  13246 non-null  object 
 4   bath        13246 non-null  float64
 5   price       13246 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 621.0+ KB


In [5]:
df_new["size"].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [6]:

# Feature Engineering (for simplicity, let's just encode 'location' and 'size')
df_new['bhk'] = df_new['size'].apply(lambda x: int(x.split()[0]))
df_new = df_new.drop(columns='size')
df_new.head()



,Unnamed: 0,location,total_sqft,bath,price,bhk
0,0,Electronic City Phase II,1056,2.0,39.07,2
1,1,Chikka Tirupathi,2600,5.0,120.00,4
2,2,Uttarahalli,1440,2.0,62.00,3
3,3,Lingadheeranahalli,1521,3.0,95.00,3
4,4,Kothanur,1200,2.0,51.00,2


In [7]:

df_new[["bhk"]]=df_new[["bhk"]].astype(int)


In [8]:

# One-hot encoding for 'location'
df_new = pd.get_dummies(df_new, columns=['location'], drop_first=True)



In [9]:
df_new.head()

,Unnamed: 0,total_sqft,bath,price,bhk,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,location_ Devarabeesana Halli,location_ Devarachikkanahalli,...,location_rr nagar,location_sankeswari,location_sapthagiri Layout,location_sarjapura main road,location_singapura paradise,location_t.c palya,location_tc.palya,location_vinayakanagar,"location_white field,kadugodi",location_whitefiled
0,0,1056,2.0,39.07,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2600,5.0,120.00,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1440,2.0,62.00,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1521,3.0,95.00,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1200,2.0,51.00,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_new.to_csv('Final_Bengaluru_new_cleaned.csv')




In [11]:
df_new["total_sqft"].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [12]:
df_new[df_new['total_sqft'].isin(["3Cents"])]

,Unnamed: 0,total_sqft,bath,price,bhk,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,location_ Devarabeesana Halli,location_ Devarachikkanahalli,...,location_rr nagar,location_sankeswari,location_sapthagiri Layout,location_sarjapura main road,location_singapura paradise,location_t.c palya,location_tc.palya,location_vinayakanagar,"location_white field,kadugodi",location_whitefiled
5419,5453,3Cents,6.0,75.0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Function to convert different units to square feet
def convert_to_sqft(value):
    if 'Sq. Yards' in value:
        return float(value.split('Sq.')[0]) * 9  # 1 Sq. Yard = 9 Sq. Feet
    elif 'Sq. Meter' in value:
        return float(value.split('Sq.')[0]) * 10.7639  # 1 Sq. Meter = 10.7639 Sq. Feet
    elif 'Acres' in value:
        return float(value.split('Acres')[0]) * 43560  # 1 Acre = 43560 Sq. Feet
    elif 'Perch' in value:
        return float(value.split('Perch')[0]) * 272.25  # 1 Perch = 272.25 Sq. Feet
    elif 'Guntha' in value:
        return float(value.split('Guntha')[0]) * 1089  # 1 Guntha = 1089 Sq. Feet
    elif 'Grounds' in value:
        return float(value.split('Grounds')[0]) * 2400  # 1 Ground = 2400 Sq. Feet
    elif 'Cents' in value:
        return float(value.split('Cents')[0]) * 435.6  # 1 Cent = 435.6 Sq. Feet
    else:
        return float(value)

# Function to process the 'total_sqft' column
def process_total_sqft(value):
    if '-' in value:
        # Split the range and calculate the average
        low, high = map(float, re.split('-', value))
        return (low + high) / 2
    else:
        # Convert units to square feet and return the value
        return convert_to_sqft(value)


In [14]:

# Apply the function to the 'total_sqft' column
df_new['total_sqft_processed'] = df_new['total_sqft'].apply(process_total_sqft)
df_new = df_new.drop(columns='total_sqft')


In [15]:

df_new.columns


Index(['Unnamed: 0', 'bath', 'price', 'bhk', 'location_ Banaswadi',
       'location_ Basavangudi', 'location_ Bhoganhalli',
       'location_ Devarabeesana Halli', 'location_ Devarachikkanahalli',
       'location_ Electronic City',
       ...
       'location_sankeswari', 'location_sapthagiri Layout',
       'location_sarjapura main road', 'location_singapura paradise',
       'location_t.c palya', 'location_tc.palya', 'location_vinayakanagar',
       'location_white field,kadugodi', 'location_whitefiled',
       'total_sqft_processed'],
      dtype='object', length=1308)

In [16]:

df_new.head()


,Unnamed: 0,bath,price,bhk,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,location_ Devarabeesana Halli,location_ Devarachikkanahalli,location_ Electronic City,...,location_sankeswari,location_sapthagiri Layout,location_sarjapura main road,location_singapura paradise,location_t.c palya,location_tc.palya,location_vinayakanagar,"location_white field,kadugodi",location_whitefiled,total_sqft_processed
0,0,2.0,39.07,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1056.0
1,1,5.0,120.00,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2600.0
2,2,2.0,62.00,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1440.0
3,3,3.0,95.00,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1521.0
4,4,2.0,51.00,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1200.0


In [17]:

df_new[df_new['total_sqft_processed'].isin(['1005.03 - 1252.49'])]


,Unnamed: 0,bath,price,bhk,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,location_ Devarabeesana Halli,location_ Devarachikkanahalli,location_ Electronic City,...,location_sankeswari,location_sapthagiri Layout,location_sarjapura main road,location_singapura paradise,location_t.c palya,location_tc.palya,location_vinayakanagar,"location_white field,kadugodi",location_whitefiled,total_sqft_processed


In [18]:
# Split the dataset into features (X) and target (y)
X = df_new.drop(columns=['price'])
y = df_new['price']


In [19]:
# Split the dataset into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:

# Create an instance of the SVR classifier
svr = SVR(kernel='linear')

# Train the classifier on the training data
svr.fit(X_train, y_train)

# Predict the labels of the testing data
y_pred = svr.predict(X_test)


In [22]:

# Evaluate the performance using MSE, MAE, and R-squared
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")


Mean Squared Error: 15729.306184602428
Mean Absolute Error: 42.79766589146696
R-squared: -0.12101531531966558


In [23]:

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1],
    'kernel': ['poly', 'rbf'],
    'degree': [2, 3],  # Only relevant for 'poly' kernel
    'gamma': ['scale', 'auto']
}


In [24]:

# Create a GridSearchCV instance
grid_search = GridSearchCV(SVR(), param_grid, cv=2, scoring='neg_mean_squared_error',refit= True , verbose=3)


In [25]:
# Fit the GridSearchCV instance on the training data
grid_search.fit(X_train, y_train)


Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV 1/2] END C=0.1, degree=2, gamma=scale, kernel=poly;, score=-26999.040 total time=  29.1s
[CV 2/2] END C=0.1, degree=2, gamma=scale, kernel=poly;, score=-28045.466 total time=  28.8s
[CV 1/2] END C=0.1, degree=2, gamma=scale, kernel=rbf;, score=-26918.123 total time=  41.7s
[CV 2/2] END C=0.1, degree=2, gamma=scale, kernel=rbf;, score=-27890.064 total time=  41.4s
[CV 1/2] END C=0.1, degree=2, gamma=auto, kernel=poly;, score=-27005.437 total time=  29.0s
[CV 2/2] END C=0.1, degree=2, gamma=auto, kernel=poly;, score=-28050.805 total time=  28.8s
[CV 1/2] END C=0.1, degree=2, gamma=auto, kernel=rbf;, score=-26926.584 total time=  41.4s
[CV 2/2] END C=0.1, degree=2, gamma=auto, kernel=rbf;, score=-27899.148 total time=  41.5s
[CV 1/2] END C=0.1, degree=3, gamma=scale, kernel=poly;, score=-26983.375 total time=  29.2s
[CV 2/2] END C=0.1, degree=3, gamma=scale, kernel=poly;, score=-28030.775 total time=  28.9s
[CV 1/2] END C=0.

GridSearchCV(cv=2, estimator=SVR(),
             param_grid={'C': [0.1, 1], 'degree': [2, 3],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'rbf']},
             scoring='neg_mean_squared_error', verbose=3)

In [26]:

# Print the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)


Best parameters found:  {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best cross-validation score:  26831.288363788713


In [27]:
# Use the best estimator found by GridSearchCV
best_svr = grid_search.best_estimator_


In [28]:

# Train the tuned classifier on the entire dataset
best_svr.fit(X, y)



SVR(C=1, degree=2)

In [29]:
# Save the trained classifier to a file
import joblib
joblib.dump(best_svr, 'best_svr_model.pkl')


['best_svr_model.pkl']